In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import time

In [2]:
news_df = pd.read_csv('./data/fake_or_real_news.csv')

In [3]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.7, stop_words='english')

In [4]:
news_df.loc[news_df['label'] == 'REAL', 'label']= 1
news_df.loc[news_df['label'] == 'FAKE', 'label'] = 0

In [5]:
X = news_df['text']
y = news_df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [6]:
tfidf_train = tfidf_vectorizer.fit_transform(X_train)

In [7]:
tfidf_test = tfidf_vectorizer.transform(X_test)

In [8]:
tfidf_train.shape

(4244, 56801)

## 1st classification method: Multinomial Naive Bayes

In [9]:
naive_bayes_clf = MultinomialNB()

In [10]:
start_time = time.time()
naive_bayes_clf.fit(tfidf_train, y_train)
end_time = time.time()
print('Time to train the model: ', (end_time - start_time))

Time to train the model:  0.01807856559753418


In [11]:
y_pred = naive_bayes_clf.predict(tfidf_test)

In [12]:
score = metrics.accuracy_score(y_true=y_test, y_pred=y_pred)

In [13]:
print("accuracy:   %0.3f" % score)
conf_matrix = metrics.confusion_matrix(y_test, y_pred, labels=[0, 1])

accuracy:   0.815


In [15]:
f_score = metrics.f1_score(y_true=y_test, y_pred=y_pred)
print('F score: ', f_score)

F score:  0.838358458961


In [23]:
print(conf_matrix)

[[ 704  367]
 [  19 1001]]


In [25]:
conf_matrix_df = pd.DataFrame(conf_matrix, 
                              index = ['Actual Fake News: 0', 'Actual Real News: 1'], 
                              columns = ['Predicted Fake News: 0', 'Predicted Real News: 1'])
conf_matrix_df

,Predicted Fake News: 0,Predicted Real News: 1
Actual Fake News: 0,704,367
Actual Real News: 1,19,1001


Using Multinomial Naive Bayes, out of 1071 fake news descriptions, 704 were correctly classified as fake.
Conversely, out of 1020 real news descriptions, 1001 were correctly identified as real.

## 2nd classification method: Logistic Regression

In [16]:
log_reg_clf = LogisticRegression()

In [17]:
start_time = time.time()
log_reg_clf.fit(tfidf_train, y_train)
end_time = time.time()
print('Time to train the model: ', (end_time - start_time))

Time to train the model:  0.2932591438293457


In [18]:
log_reg_y_pred = log_reg_clf.predict(tfidf_test)

In [19]:
log_reg_score = metrics.accuracy_score(y_true=y_test, y_pred=log_reg_y_pred)

In [20]:
print("accuracy:   %0.3f" % log_reg_score)
conf_matrix = metrics.confusion_matrix(y_test, log_reg_y_pred, labels=[0, 1])

accuracy:   0.915


In [22]:
f_score = metrics.f1_score(y_true=y_test, y_pred=log_reg_y_pred)
print('F score: ', f_score)

F score:  0.91046277666


In [29]:
print(conf_matrix)

[[1008   63]
 [ 115  905]]


In [30]:
conf_matrix_df = pd.DataFrame(conf_matrix, 
                              index = ['Actual Fake News: 0', 'Actual Real News: 1'], 
                              columns = ['Predicted Fake News: 0', 'Predicted Real News: 1'])
conf_matrix_df

,Predicted Fake News: 0,Predicted Real News: 1
Actual Fake News: 0,1008,63
Actual Real News: 1,115,905


Using Logistic Regression, out of 1071 fake news descriptions, 1008 were correctly classified as fake.
Conversely, out of 1020 real news descriptions, 905 were correctly identified as real.